In [1]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Suicide_Detection.csv', engine='python')


In [3]:
#Cambiamos la clase a dummy y achicamos el dataset para poder entrenarlo


In [4]:
df = pd.get_dummies(df, columns = ['class'], drop_first = True)

In [5]:
df = df.iloc[:,1:]

In [6]:
df = df.iloc[220000:]


In [7]:
#Importamos librerias para trabajar sobre text mining
from nltk import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
# Defining Features Matrix
X = df.drop(['class_suicide'], axis=1)
X.head()

,text
220000,My inability to sleep... ...It is frightening....
220001,I just woke up at 2 pm hows your day going?
220002,I want more snap streaks so I can see cute boi...
220003,I read an interesting article on notesI starte...
220004,The horrors of the climbing unit in Gym class ...


In [9]:
# Define Target
y = df['class_suicide']
y.head()

220000    0
220001    0
220002    0
220003    1
220004    0
Name: class_suicide, dtype: uint8

In [10]:
# Separate training and testing sets, stratifying by class
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [11]:
#Vamos a limpiar los datos de train y test
#Usamos Tokennizer que elimine los signos de puntuación y tags html
#Hacemos stemming para obtener las raices de las palabras en minusculas
#Eliminamos stopwords

In [12]:
def clean_reddit(text, tokenizer, stemmer, stopwords):
    
    #tokens (eliminamos todos los signos de puntuación)
    words = tokenizer.tokenize(text)
    
    #Stemming : raiz y minusculas:
    stem_words = [stemmer.stem(x) for x in words]
    
    #eliminamos stopwords (ya pasaron por stem)
    clean_words = [x for x in stem_words if x not in stopwords]
    
    result = " ".join(clean_words)
    
    return(result)

In [13]:
#Importing for stepwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Equipo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
#Eliminamos signos de puntuacion

tokenizer = RegexpTokenizer(r"\w+")

englishStemmer = SnowballStemmer("english")
stopwords_en = stopwords.words('english')
stopwords_en_stem = [englishStemmer.stem(x) for x in stopwords_en]

In [15]:
clean_train = [clean_reddit(x, tokenizer, englishStemmer, stopwords_en_stem) for x in X_train.text]

In [16]:
clean_test = [clean_reddit(x, tokenizer, englishStemmer, stopwords_en_stem) for x in X_test.text]

In [17]:
#CountVectorizer para transformar los datos de train y test

count_vectorizer = CountVectorizer()
count_vectorizer.fit(clean_train)
X_train_sparse = count_vectorizer.transform(clean_train)
X_test_sparse = count_vectorizer.transform(clean_test)

In [18]:
X_train = pd.DataFrame(X_train_sparse.todense(), 
             columns = count_vectorizer.get_feature_names())

In [19]:
X_test = pd.DataFrame(X_test_sparse.todense(), 
             columns = count_vectorizer.get_feature_names()) 

In [21]:
#Entrenamos decision tree por defecto

In [24]:
from sklearn import tree


In [25]:
model_tree = tree.DecisionTreeClassifier(criterion='gini')

In [26]:
fit_tree = model_tree.fit(X_train, y_train)

In [27]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [30]:
predict_tree_cat = fit_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, predict_tree_cat)
print(accuracy_tree)
conf_mat_tree = confusion_matrix(y_test, predict_tree_cat)
print(conf_mat_tree)

0.8370321298443193
[[1274  240]
 [ 252 1253]]


In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predict_tree_cat))

              precision    recall  f1-score   support

           0       0.83      0.84      0.84      1514
           1       0.84      0.83      0.84      1505

    accuracy                           0.84      3019
   macro avg       0.84      0.84      0.84      3019
weighted avg       0.84      0.84      0.84      3019

